In [2]:
from elasticsearch import Elasticsearch, helpers
import threading
import json
import re
from moojing import Exporter
import pandas as pd
pd.set_option("max_colwidth",100)
from pymongo import *

import arrow

ls1=[]
ls1_comment_id=[]
ls1_item_id=[]
ls1_ts=[]
ls1_skuid=[]

xxx=['672132332115','673015568231','673016384363']
 
for i in xxx:
    es = Elasticsearch('http://10.19.12.67:9200')
    index = ["polarity_tmcomment_v4_1_p1*", 'polarity_tmcomment_v4_1_p2*', 'polarity_tmcomment_v4_1_p3*', "polarity_tmcomment_v4_3*","polarity_tmcomment_v4_4*","polarity_tmcomment_v4_3*"]

    body = {"query": {"terms": {"item_id": [i]}}, 'sort': 'ctime', "track_total_hits": True}
    res = es.search(index=index, body=body, size=10000, request_timeout=60)

    for r in res['hits']['hits']:
        ls1.append(r['_source']['content'])
        ls1_comment_id.append(r['_source']['comment_id'])
        ls1_item_id.append(r['_source']['item_id'])
        ls1_ts.append(r['_source']['ctime'])
        ls1_skuid.append(r['_source']['skuid'])
print(len(ls1))

186


In [3]:
import xlwt
f = xlwt.Workbook('utf-8') #设置工作簿编码
sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) #创建sheet工作表
sheet1.write(0,0,'item_id') #写入数据参数对应 行, 列, 值
sheet1.write(0,1,'comment_id') #写入数据参数对应 行, 列, 值
sheet1.write(0,2,'ts') #写入数据参数对应 行, 列, 值
sheet1.write(0,3,'full_comment') #写入数据参数对应 行, 列, 值
sheet1.write(0,4,'sku_id') #写入数据参数对应 行, 列, 值

for i in range(len(ls1)):
    sheet1.write(i+1,0,ls1_item_id[i]) #写入数据参数对应 行, 列, 值
    sheet1.write(i+1,1,ls1_comment_id[i]) #写入数据参数对应 行, 列, 值
    sheet1.write(i+1,2,ls1_ts[i]) #写入数据参数对应 行, 列, 值
    sheet1.write(i+1,3,ls1[i]) #写入数据参数对应 行, 列, 值
    sheet1.write(i+1,4,ls1_skuid[i]) #写入数据参数对应 行, 列, 值

f.save('data.xls')#保存.xls到当前工作目录

In [4]:

# -*- coding: utf8 -*-
import json
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdknlp_automl.request.v20191111 import RunPreTrainServiceRequest
import sys
import pandas as pd
import json
def aliyun_model(comment, aspect):
    # Initialize AcsClient instance
    client = AcsClient(
      "LTAI5tBDbYayt5mi7qc34YAt",
      "FgmiiRV3D330nWMlTdklXsacd6GyXJ",
      "cn-hangzhou"
    );
    content =  comment
    domain = aspect
    input_params = {
        "input": {
            "content": content,
            "domain": domain,
            "entity": True
        }
    }
    # Initialize a request and set parameters
    request = RunPreTrainServiceRequest.RunPreTrainServiceRequest()
    request.set_ServiceName('ReviewAnalysis')
    request.set_PredictContent(json.dumps(input_params))
    # Print response
    response = client.do_action_with_exception(request)
    #print(response)
    resp_obj = json.loads(response.decode('utf-8'))
    predict_result = json.loads(resp_obj['PredictResult'])
    return json.dumps(predict_result['data'])


In [5]:
def get_aliyun(df, aspect):
    res_list = []
    counter = 0
    for index, comment in df.iterrows():
        print(counter)
        #print(comment['comment'])
        try:
            res = aliyun_model(comment['comment'], aspect)
            res_list.append(json.loads(res))
            counter += 1
        except:
            print('Exc')
            print(traceback.print_exc())
            time.sleep(1)
    print(counter)
    return res_list

In [6]:
#第11行加了一句count += 1
def formed(res_list):
    columns = ['father','clause', 'aspect_term', 'opinion_term', 'positive', 'negative', 'category','normedOpinionTerm']
    aliyun_polar = pd.DataFrame(columns = columns)
    count = 0
    for res in res_list:
        try:
            aspectItem = res['aspectItem']
        except:
           # print(traceback.print_exc())
            count += 1
            print(res)
            continue
        for clauseItem in aspectItem:
            try:
                clause = clauseItem['clause']
                aspt = []
                opt = []
                norm=[]
            except:
                print(traceback.print_exc())
                print(aspectItem)
                continue
            for terms in clauseItem['terms']:
                try:
                    aspectTerm = terms['aspectTerm']
                    opinionTerm = terms['opinionTerm']
                    normedOpinionTerm = terms['normedOpinionTerm']
                    norm.append(normedOpinionTerm)
                    aspt.append(aspectTerm)
                    opt.append(opinionTerm)
                except:
                    print(traceback.print_exc())
                    continue
            aspterm = ' '.join(aspt)
            opterm = ' '.join(opt)
            normerm = ' '.join(norm)
            positiveProb = clauseItem['positiveProb']
            aspectCategory = clauseItem['aspectCategory']
            negativeProb = clauseItem['negativeProb']
            curdf = [[count,clause,aspterm,opterm,positiveProb,negativeProb,aspectCategory,normerm]]
            curdf = pd.DataFrame(curdf, columns = columns)
            aliyun_polar = aliyun_polar.append(curdf)
        count += 1
    return aliyun_polar

In [7]:

data_wan = pd.read_excel('data_aoeo.xls')
data_wan['comment']=data_wan['full_comment']

In [8]:
data_wan

,item_id,comment_id,ts,full_comment,sku_id,comment
0,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,4840371888249,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评
1,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,4840371888249,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！
2,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,4840371888249,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。
3,672132332115,1.174024e+12,2022-05-03,物流特别快 下午发货 第二天早上就收到了 一直期待新品～上脸试用一点都不粘腻 我是干敏皮 补水保湿效果很好 用了没有过敏现象 家里人油皮 用了也说一点不粘腻 会一直支持！！希望越做越好！！,4840371888249,物流特别快 下午发货 第二天早上就收到了 一直期待新品～上脸试用一点都不粘腻 我是干敏皮 补水保湿效果很好 用了没有过敏现象 家里人油皮 用了也说一点不粘腻 会一直支持！！希望越做越好！！
4,672132332115,1.173593e+12,2022-05-03,大干皮的爱，之前买过洗面奶和面膜，很舒服好好用，第二天起床皮肤嫩嫩的，一整天状态都很好，搞活动又买了水乳套装，竟然送了精华，还送了面膜和卸妆巾，太赞了，国货走起！！！,4840371888249,大干皮的爱，之前买过洗面奶和面膜，很舒服好好用，第二天起床皮肤嫩嫩的，一整天状态都很好，搞活动又买了水乳套装，竟然送了精华，还送了面膜和卸妆巾，太赞了，国货走起！！！
5,672132332115,1.174447e+12,2022-05-03,质地手感：快递非常快，收到后迫不及待打开，质感不错，效果也是杠杠的！,4840371888249,质地手感：快递非常快，收到后迫不及待打开，质感不错，效果也是杠杠的！
6,672132332115,1.174131e+12,2022-05-04,宝贝很好，送的赠品很多，很喜欢，一直用这款洗面奶，现在用水和乳，看着很不错。,4840371888249,宝贝很好，送的赠品很多，很喜欢，一直用这款洗面奶，现在用水和乳，看着很不错。
7,672132332115,1.174189e+12,2022-05-04,整体评价：用后感觉特别好用不油腻，上妆服帖，很多同事都说我的皮肤好，值得信赖的一款产品，我的家人都在用都说好,4840371888249,整体评价：用后感觉特别好用不油腻，上妆服帖，很多同事都说我的皮肤好，值得信赖的一款产品，我的家人都在用都说好
8,672132332115,1.173323e+12,2022-05-04,保湿控油效果：脸颊千 鼻子油 混干皮 吸收效果：清爽不油，这一款真是惊也，我保证会回购啊真的绝 最主要是我喜欢的泵头形式不会乱滴 ！会回购 ！我脸上有点痘印痘痘继续用希望会有改善 总之就是爱！,4840371888249,保湿控油效果：脸颊千 鼻子油 混干皮 吸收效果：清爽不油，这一款真是惊也，我保证会回购啊真的绝 最主要是我喜欢的泵头形式不会乱滴 ！会回购 ！我脸上有点痘印痘痘继续用希望会有改善 总之就是爱！
9,672132332115,1.174093e+12,2022-05-04,有一说一山茶花水乳的包装真的好高级，玻璃瓶子质地显得bug很高，新品预售提前付了意向金还送了一盒面膜，小O发带哦，嘿嘿😁，小盒子又满当当啦,4840371888249,有一说一山茶花水乳的包装真的好高级，玻璃瓶子质地显得bug很高，新品预售提前付了意向金还送了一盒面膜，小O发带哦，嘿嘿😁，小盒子又满当当啦


In [18]:
#result=get_aliyun(data_wan[:5000], 'beauty')
#result_formed=formed(result)

In [9]:
#保存数据
#result_formed.to_csv("aoeo第二次.csv")
result_formed= pd.read_csv("aoeo第二次.csv")

In [10]:
result_formed

,Unnamed: 0,father,clause,aspect_term,opinion_term,positive,negative,category,normedOpinionTerm
0,0,0,整体评价：发货很快,发货,很快,1.000,0.000,物流,很快
1,0,0,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,送的东西,很惊艳,1.000,0.000,赠品,很惊艳
2,0,0,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,洗面奶,绝绝子,0.971,0.029,整体,绝绝子
3,0,1,好评好评～极力推荐,NaN,好评好评,1.000,0.000,整体,好评好评
4,0,1,好评好评～极力推荐,NaN,极力推荐,1.000,0.000,其他,极力推荐
5,0,1,整体评价：包装好好看吖,包装,好好看,1.000,0.000,包装,很好看
6,0,1,洗的很干净,洗的,很干净,1.000,0.000,使用效果,很干净
7,0,2,肤感好、还保湿,NaN,保湿,1.000,0.000,补水保湿效果,保湿
8,0,2,有一股淡淡的香味,NaN,有一股淡淡的香味,1.000,0.000,异味/气味,有一股淡淡的香味
9,0,2,挺喜欢的,NaN,挺喜欢,1.000,0.000,整体,喜欢


In [11]:

result_formed['item_id']=''
result_formed['comment_id']=''
result_formed['ts']=''
result_formed['full_comment']=''
result_formed['clasue']=result_formed['clause']
result_formed['aspect']=result_formed['category'] 
result_formed['polarity']=''
result_formed['opinionTerm']=result_formed['opinion_term']
item_id=[]
comment_id=[]
ts=[]
full_comment=[]
polarity=[]
for i in range(len(result_formed['item_id'])):
    full_comment.append(data_wan['full_comment'][list(result_formed['father'])[i]])
    item_id.append(data_wan['item_id'][list(result_formed['father'])[i]])
    comment_id.append(data_wan['comment_id'][list(result_formed['father'])[i]])
    ts.append(data_wan['ts'][list(result_formed['father'])[i]])
    if (float(list(result_formed['negative'])[i])>float(list(result_formed['positive'])[i])):
        polarity.append('negative')
    else:
        polarity.append('positive')

result_formed['item_id']=item_id
result_formed['comment_id']=comment_id
result_formed['ts']=ts
result_formed['full_comment']=full_comment
result_formed['polarity']=polarity

In [12]:
result_formed

,Unnamed: 0,father,clause,aspect_term,opinion_term,positive,negative,category,normedOpinionTerm,item_id,comment_id,ts,full_comment,clasue,aspect,polarity,opinionTerm
0,0,0,整体评价：发货很快,发货,很快,1.000,0.000,物流,很快,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,整体评价：发货很快,物流,positive,很快
1,0,0,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,送的东西,很惊艳,1.000,0.000,赠品,很惊艳,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,赠品,positive,很惊艳
2,0,0,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,洗面奶,绝绝子,0.971,0.029,整体,绝绝子,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,整体,positive,绝绝子
3,0,1,好评好评～极力推荐,NaN,好评好评,1.000,0.000,整体,好评好评,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,好评好评～极力推荐,整体,positive,好评好评
4,0,1,好评好评～极力推荐,NaN,极力推荐,1.000,0.000,其他,极力推荐,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,好评好评～极力推荐,其他,positive,极力推荐
5,0,1,整体评价：包装好好看吖,包装,好好看,1.000,0.000,包装,很好看,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,整体评价：包装好好看吖,包装,positive,好好看
6,0,1,洗的很干净,洗的,很干净,1.000,0.000,使用效果,很干净,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,洗的很干净,使用效果,positive,很干净
7,0,2,肤感好、还保湿,NaN,保湿,1.000,0.000,补水保湿效果,保湿,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,肤感好、还保湿,补水保湿效果,positive,保湿
8,0,2,有一股淡淡的香味,NaN,有一股淡淡的香味,1.000,0.000,异味/气味,有一股淡淡的香味,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,有一股淡淡的香味,异味/气味,positive,有一股淡淡的香味
9,0,2,挺喜欢的,NaN,挺喜欢,1.000,0.000,整体,喜欢,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,挺喜欢的,整体,positive,挺喜欢


In [13]:
result_end=result_formed[['item_id','comment_id','ts','full_comment','clasue','aspect','polarity','opinionTerm','normedOpinionTerm']]


In [14]:
result_end=result_end.reset_index(drop=True)

In [15]:
result_end

,item_id,comment_id,ts,full_comment,clasue,aspect,polarity,opinionTerm,normedOpinionTerm
0,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,整体评价：发货很快,物流,positive,很快,很快
1,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,赠品,positive,很惊艳,很惊艳
2,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,整体,positive,绝绝子,绝绝子
3,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,好评好评～极力推荐,整体,positive,好评好评,好评好评
4,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,好评好评～极力推荐,其他,positive,极力推荐,极力推荐
5,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,整体评价：包装好好看吖,包装,positive,好好看,很好看
6,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,洗的很干净,使用效果,positive,很干净,很干净
7,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,肤感好、还保湿,补水保湿效果,positive,保湿,保湿
8,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,有一股淡淡的香味,异味/气味,positive,有一股淡淡的香味,有一股淡淡的香味
9,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,挺喜欢的,整体,positive,挺喜欢,喜欢


In [17]:
import pandas as pd
result_end_2=pd.DataFrame(columns=result_end.columns)
for i in range(len(result_end)):
    x=result_end['normedOpinionTerm'][i]
    xx=str(x).split(' ')
    for k in xx:
        lis={}
        for l in result_end.columns:
            if l=='normedOpinionTerm':
                lis[l]=k
            else:
                lis[l]=result_end[l][i]
        result_end_2=result_end_2.append(lis , ignore_index=True)
        print(len(result_end_2))
result_end_2.to_csv("AOEOresult.csv",encoding = 'utf_8_sig')

In [91]:
result_end_2

,item_id,comment_id,ts,full_comment,clasue,aspect,polarity,opinionTerm,normedOpinionTerm
0,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,整体评价：发货很快,物流,positive,很快,很快
1,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,赠品,positive,很惊艳,很惊艳
2,672132332115,1.173590e+12,2022-05-03,整体评价：发货很快，付了尾款马上就发货了，最重要和客服小姐姐申请到了一个眼霜小样，这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子，这次的新品还没用，等效果追评,这次送的东西真的很惊艳 质地手感：洗面奶真的是绝绝子,整体,positive,绝绝子,绝绝子
3,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,好评好评～极力推荐,整体,positive,好评好评,好评好评
4,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,好评好评～极力推荐,其他,positive,极力推荐,极力推荐
5,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,整体评价：包装好好看吖,包装,positive,好好看,很好看
6,672132332115,1.174039e+12,2022-05-03,整体评价：包装好好看吖！ 保湿控油效果：很适合我的皮肤，洗的很干净！好评好评～极力推荐！,洗的很干净,使用效果,positive,很干净,很干净
7,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,肤感好、还保湿,补水保湿效果,positive,保湿,保湿
8,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,有一股淡淡的香味,异味/气味,positive,有一股淡淡的香味,有一股淡淡的香味
9,672132332115,1.174443e+12,2022-05-03,整体评价：发的是顺风快递，第二天就拿到了。商家很用心，包装盒都是很好看的。回到家就拆开使用了下，肤感好、还保湿，有一股淡淡的香味，挺喜欢的。,挺喜欢的,整体,positive,挺喜欢,喜欢
